## PreProcessing and Modeling

In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# bring in testing data
test_df = pd.read_csv('DB/test_sample.csv', index_col='Unnamed: 0')
test_df = test_df.dropna()

# bring in flights data
flights_df = pd.read_csv('DB/flights_data.csv', index_col='Unnamed: 0')

In [3]:
# find testing data features
feature_cols = list(test_df.columns)
feature_cols.append('arr_delay')

# create base training features from existing testing features
X = flights_df[feature_cols]
X = X.dropna()

In [4]:
# find numeric and categorical features
cols = X.columns
num_cols = X._get_numeric_data().columns
cat_cols = list(set(cols) - set(num_cols))

# remove redundant numeric columns
final_num_cols = list(num_cols)
final_num_cols.remove('op_carrier_fl_num')
final_num_cols.remove('flights')

# remove redundant categorical columns
final_cat_cols = ['mkt_unique_carrier', 'fl_date', 'tail_num', 'branded_code_share']

# combine final features
final_features = final_num_cols + final_cat_cols

X = X[final_features]

# convert fl_date feature into datetime
X['fl_date'] = pd.to_datetime(X['fl_date'])

# separate datetime into date features
X['year'] = X['fl_date'].dt.year
X['month'] = X['fl_date'].dt.month
X['week'] = X['fl_date'].dt.isocalendar().week
X['day'] = X['fl_date'].dt.day
X['day_of_week'] = X['fl_date'].dt.dayofweek

# reset index for collaborative data sorting structure
X = X.reset_index()
X.index.name = 'order'
X = X.drop(columns=['index'])

# drop original fl_date and arr_delay columns
X = X.drop(columns=['fl_date'])

***

## Join New Features

In [5]:
# bring in feature data
taxi_and_delay = pd.read_csv('James/features_created.csv')
avg_monthly_pas = pd.read_csv('Riley/avg_monthly_pas.csv')


In [6]:
taxi_and_delay.head()

,Unnamed: 0,order,distance,crs_dep_time,crs_arr_time,mkt_unique_carrier,mean_taxi_out/time,mean_taxi_in/time,mean_dep_delay/time,mean_arr_delay/time,mean_dep_delay/distance,mean_arr_delay/distance,mean_dep_delay/carrier,mean_arr_delay/carrier
0,0,0,733.0,1300,1444,UA,17.372642,8.398230,9.557783,16.203540,17.384397,13.644793,12.628894,8.687159
1,1,1,1075.0,630,854,UA,16.813246,9.232143,4.561346,5.321429,8.236994,0.011561,12.628894,8.687159
2,2,2,488.0,1500,1709,UA,18.789406,8.639053,14.756792,10.242604,13.277929,7.323288,12.628894,8.687159
3,3,3,199.0,2041,2159,UA,21.924528,7.333333,11.867925,18.743961,14.227513,6.284946,12.628894,8.687159
4,4,4,224.0,2140,2257,UA,17.312977,8.170455,14.675573,3.227273,10.140485,8.314578,12.628894,8.687159


In [7]:
# drop extra column
taxi_and_delay = taxi_and_delay.drop(columns=['Unnamed: 0'])

In [8]:
# merge taxi and delay features onto base training dataset
merge_cols = ['order',
              'mean_taxi_out/time',
              'mean_taxi_in/time',
              'mean_dep_delay/time',
              'mean_arr_delay/time',
              'mean_dep_delay/distance',
              'mean_arr_delay/distance',
              'mean_dep_delay/carrier',
              'mean_arr_delay/carrier']

X = pd.merge(X, taxi_and_delay[merge_cols], how='left', on=['order'])

In [9]:
# merge onto training DataFrame
final = pd.merge(X, avg_monthly_pas, how='left', on=['origin_airport_id','month'])

In [10]:
final = final.dropna()

In [11]:
final = final.drop(['order'], axis=1)
final.head()

,mkt_carrier_fl_num,origin_airport_id,dest_airport_id,crs_dep_time,crs_arr_time,crs_elapsed_time,distance,arr_delay,mkt_unique_carrier,tail_num,...,day_of_week,mean_taxi_out/time,mean_taxi_in/time,mean_dep_delay/time,mean_arr_delay/time,mean_dep_delay/distance,mean_arr_delay/distance,mean_dep_delay/carrier,mean_arr_delay/carrier,avg_monthly_pas
0,3501,12953,13930,1300,1444,164.0,733.0,-28.0,UA,N744YX,...,0,17.372642,8.398230,9.557783,16.203540,17.384397,13.644793,12.628894,8.687159,66470.5
1,3502,11433,12266,630,854,204.0,1075.0,1.0,UA,N640RW,...,0,16.813246,9.232143,4.561346,5.321429,8.236994,0.011561,12.628894,8.687159,108812.5
2,3503,11618,11433,1500,1709,129.0,488.0,18.0,UA,N641RW,...,0,18.789406,8.639053,14.756792,10.242604,13.277929,7.323288,12.628894,8.687159,94693.5
3,3504,11618,11278,2041,2159,78.0,199.0,32.0,UA,N722YX,...,0,21.924528,7.333333,11.867925,18.743961,14.227513,6.284946,12.628894,8.687159,94693.5
4,3505,12266,11298,2140,2257,77.0,224.0,-1.0,UA,N855RW,...,0,17.312977,8.170455,14.675573,3.227273,10.140485,8.314578,12.628894,8.687159,155988.0


In [12]:
final.shape

(156741, 25)

In [13]:
final.to_csv('final_training.csv', index=False)

***

## ML Setup

In [2]:
df = pd.read_csv('final_training.csv')

In [3]:
# set X & y
X = df.drop(columns=['arr_delay'])
y = df['arr_delay']

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 101)

In [5]:
import category_encoders as ce

encoder = ce.OrdinalEncoder(cols=['mkt_unique_carrier', 'tail_num', 'branded_code_share'])

X_train = encoder.fit_transform(X_train)

X_test = encoder.transform(X_test)

C:\Users\nikif\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [6]:
X_train.shape

(109718, 24)

In [90]:
# importing scalers and PCA
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.decomposition import PCA

In [89]:
# creating function for preprocessing
# for use with score_model function
def preprocess_data(X_train, X_test, method):
    '''
    Function to scale/process data with specified method
    Dependencies: from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
                  from sklearn.decomposition import PCA
    
    X_train = train split from train_test_split
    X_test = test split from train_test_split
    method = desired scaling/processing method as str
             select from:
             {'RobustScale','StandardScale','MinMaxScale','PCA'}
    '''
    if method == 'RobustScale':
        rs = RobustScaler()
        X_train = rs.fit_transform(X_train)
        X_test = rs.transform(X_test)
    
    elif method == 'StandardScale':
        ss = StandardScaler()
        X_train = ss.fit_transform(X_train)
        X_test = ss.transform(X_test)
    
    elif method == 'MinMaxScale':
        mm = MinMaxScaler()
        X_train = mm.fit_transform(X_train)
        X_test = mm.transform(X_test)
        
    elif method == 'PCA':
        mm = MinMaxScaler()
        X_train = mm.fit_transform(X_train)
        X_test = mm.transform(X_test)
        # pca above will pick components to get to 95% of variance explained
        pca = PCA(n_components=0.95, svd_solver='full', random_state=101)
        X_train = pca.fit_transform(X_train)
        X_test = pca.transform(X_test)

In [10]:
# importing models to work with
from xgboost import XGBRegressor # XGBoost Regression
from sklearn.neighbors import KNeighborsRegressor # KNeighbours Regression
from sklearn.tree import DecisionTreeRegressor # Decision Tree Regression
from sklearn.model_selection import GridSearchCV # GridSearch for optimizing
from sklearn import metrics as me # metrics for evaluation

In [88]:
def score_model(label, model, X_train, y_train, X_test, y_test, scaling='Base'):
    '''
    Function to score given model with given data and scaling label
    Additionally, will append scores to dictionary 'results' in format:
        '{label}_{scaling}':('train_r2','train_RMSE','test_r2','test_RMSE')
    
    Dependencies: import numpy as np
                  from sklearn import metrics as me
                  from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
                  from sklearn.decomposition import PCA
    
    label = model name label for results dictionary as str
            ex. 'XGBoost' or 'Linear Regression'
    model = model instantiated with optimal params set
    X_train, y_train = training data split used as np.array
    X_test, y_test = data split to test as np.array
    scaling = scaling/preprocessing to use on data as str
              select from:
              {'RobustScale','StandardScale','MinMaxScale','PCA','Base'}
              default = 'Base' for no scaling/reduction used
    '''
    if 'results' in globals(): # check to ensure dictionary is not over-written
        pass
    else:
        # set global dictionary to store results
        global results
        results = {}
    
    
   
    # perform scaling
    if scaling == 'Base':
        pass
    else:
        preprocess_data(X_train, X_test, scaling) 
        
        
    # fit model
    model.fit(X_train, y_train)

    # predict Train set results
    y_pred_train = model.predict(X_train)

    # predict Test set results
    y_pred = model.predict(X_test)

    # computing RMSE + R2
    rmse_train = np.sqrt(me.mean_squared_error(y_train, y_pred_train))
    rmse_test = np.sqrt(me.mean_squared_error(y_test, y_pred))
    r2_train = me.r2_score(y_train, y_pred_train)
    r2_test = me.r2_score(y_test, y_pred)
    
    # create dictionary entry
    entry = f'{label}_{scaling}'
    results[entry] = (r2_train, rmse_train, r2_test, rmse_test)
    
    # print scores
    print(f"{label} {scaling} data score:")
    print("Train R2: %f" % (r2_train))
    print("Train RMSE: %f" % (rmse_train))
    print("Test R2: %f" % (r2_test))
    print("Test RMSE: %f" % (rmse_test))

### XGBoost Regressor

In [28]:
# instantiating classifier
xgb = XGBRegressor()

# create params for GridSearch
param_grid = {'objective':['reg:squarederror'],
              'learning_rate': [0.03, 0.05, 0.07], 
              'max_depth': [5, 6, 7],
              'min_child_weight': [2, 3, 4],
              'subsample': [0.7],
              'colsample_bytree': [0.5, 0.7],
              'n_estimators': [100, 500, 1000]}



# instantiate GridSearch
gscv = GridSearchCV(estimator=xgb, param_grid=param_grid, n_jobs=-1)

# fit model
xgb_base = gscv.fit(X_train, y_train)

# get best params
print('Best parameters for Base:', xgb_base.best_params_)

Best parameters for Base: {'colsample_bytree': 0.5, 'learning_rate': 0.03, 'max_depth': 7, 'min_child_weight': 4, 'n_estimators': 500, 'objective': 'reg:squarederror', 'subsample': 0.7}


In [27]:
# setting optimal params to model
xgb1 = XGBRegressor(
    colsample_bytree=0.5, 
    learning_rate=0.03,
    max_depth=7,
    min_child_weight=4,
    n_estimators=500,
    objective='reg:squarederror',
    subsample=0.7
)

In [44]:
# running base data
score_model('XGBoost', xgb1, X_train, y_train, X_test, y_test)

XGBoost Base data score:
Train R2: 0.230241
Train RMSE: 41.779740
Test R2: 0.036613
Test RMSE: 49.048042


In [45]:
# running StandardScaler data
score_model('XGBoost', xgb1, X_train_ss, y_train, X_test_ss, y_test, 'StandardScale')

XGBoost StandardScale data score:
Train R2: 0.228117
Train RMSE: 41.837357
Test R2: 0.036203
Test RMSE: 49.058476


In [46]:
# running MinMaxScaler data
score_model('XGBoost', xgb1, X_train_mm, y_train, X_test_mm, y_test, 'MinMaxScale')

XGBoost MinMaxScale data score:
Train R2: 0.230531
Train RMSE: 41.771884
Test R2: 0.035830
Test RMSE: 49.067973


In [48]:
# running RobustScaler data
score_model('XGBoost', xgb1, X_train_rs, y_train, X_test_rs, y_test, 'RobustScale')

XGBoost RobustScale data score:
Train R2: 0.230650
Train RMSE: 41.768656
Test R2: 0.036294
Test RMSE: 49.056157


In [49]:
# running PCA data
score_model('XGBoost', xgb1, X_train, y_train, X_test, y_test, 'PCA')

XGBoost PCA data score:
Train R2: 0.230241
Train RMSE: 41.779740
Test R2: 0.036613
Test RMSE: 49.048042


### Decision Tree

In [40]:
# Create Decision Tree classifer object
dtree = DecisionTreeRegressor()

# create params for GridSearch
param_grid = {'criterion':['squared_error'],
              'max_depth': [None, 2, 3, 5, 7], 
              'min_samples_split': [2, 3, 4, 5],
              'max_features':  ['auto', 'sqrt', 'log2'],
              'splitter': ['best','random']}



# instantiate GridSearch
gscv1 = GridSearchCV(estimator=dtree, param_grid=param_grid, n_jobs=-1)

# fit model
tree_base = gscv1.fit(X_train, y_train)
tree_ss = gscv1.fit(X_train_ss, y_train)
tree_mm = gscv1.fit(X_train_mm, y_train)
tree_rs = gscv1.fit(X_train_rs, y_train)

# get best params
print('Best parameters for Base:', tree_base.best_params_)
print('Best parameters for StandardScale:', tree_ss.best_params_)
print('Best parameters for MinMaxScale:', tree_mm.best_params_)
print('Best parameters for RobustScale:', tree_rs.best_params_)

Best parameters for Base: {'criterion': 'squared_error', 'max_depth': 5, 'max_features': 'auto', 'min_samples_split': 4, 'splitter': 'random'}
Best parameters for StandardScale: {'criterion': 'squared_error', 'max_depth': 5, 'max_features': 'auto', 'min_samples_split': 4, 'splitter': 'random'}
Best parameters for MinMaxScale: {'criterion': 'squared_error', 'max_depth': 5, 'max_features': 'auto', 'min_samples_split': 4, 'splitter': 'random'}
Best parameters for RobustScale: {'criterion': 'squared_error', 'max_depth': 5, 'max_features': 'auto', 'min_samples_split': 4, 'splitter': 'random'}


In [51]:
# setting optimal params to model
dtree1 = DecisionTreeRegressor(
    criterion= 'squared_error', 
    max_depth= 5, 
    max_features='auto', 
    min_samples_split= 4, 
    splitter='random'
)

In [53]:
# running Base data
score_model('DecisionTree', dtree1, X_train, y_train, X_test, y_test)

DecisionTree Base data score:
Train R2: 0.012217
Train RMSE: 47.328110
Test R2: 0.011627
Test RMSE: 49.680023


In [54]:
# running StandardScale data
score_model('DecisionTree', dtree1, X_train, y_train, X_test, y_test, 'StandardScale')

DecisionTree StandardScale data score:
Train R2: 0.015202
Train RMSE: 47.256544
Test R2: 0.015045
Test RMSE: 49.594042


In [55]:
# running MinMaxScale data
score_model('DecisionTree', dtree1, X_train, y_train, X_test, y_test, 'MinMaxScale')

DecisionTree MinMaxScale data score:
Train R2: 0.014639
Train RMSE: 47.270066
Test R2: 0.011949
Test RMSE: 49.671934


In [56]:
# running RobustScale data
score_model('DecisionTree', dtree1, X_train, y_train, X_test, y_test, 'RobustScale')

DecisionTree RobustScale data score:
Train R2: 0.023159
Train RMSE: 47.065257
Test R2: 0.011841
Test RMSE: 49.674638


In [57]:
# running PCA data
score_model('DecisionTree', dtree1, X_train, y_train, X_test, y_test, 'PCA')

DecisionTree PCA data score:
Train R2: 0.014769
Train RMSE: 47.266947
Test R2: 0.014489
Test RMSE: 49.608029


### KNN Regression

In [58]:
# Create KNRegressor classifer object
knr = KNeighborsRegressor()

In [61]:
# running Standard data
score_model('KNeighbours', knr, X_train, y_train, X_test, y_test, 'StandardScale')

KNeighbours StandardScale data score:
Train R2: 0.233446
Train RMSE: 41.692673
Test R2: -0.147226
Test RMSE: 53.523667


In [82]:
# running MinMaxScale data
score_model('KNeighbours', knr, X_train, y_train, X_test, y_test, 'MinMaxScale')

KNeighbours MinMaxScale data score:
Train R2: 0.233446
Train RMSE: 41.692673
Test R2: -0.147226
Test RMSE: 53.523667


In [83]:
# running RobustScale data
score_model('KNeighbours', knr, X_train, y_train, X_test, y_test, 'RobustScale')

KNeighbours RobustScale data score:
Train R2: 0.233446
Train RMSE: 41.692673
Test R2: -0.147226
Test RMSE: 53.523667


In [84]:
# running PCA data
score_model('KNeighbours', knr, X_train, y_train, X_test, y_test, 'PCA')

KNeighbours PCA data score:
Train R2: 0.233446
Train RMSE: 41.692673
Test R2: -0.147226
Test RMSE: 53.523667


### Comparing results

##### Obtaining top two results from models:

In [85]:
# creating df
results_df = pd.DataFrame.from_dict(results, orient='index',
                       columns=['Train r2', 'Train RMSE', 'Test r2', 'Test RMSE'])

# checking
results_df

,Train r2,Train RMSE,Test r2,Test RMSE
XGBoost_Base,0.230241,41.779740,0.036613,49.048042
XGBoost_StandardScale,0.228117,41.837357,0.036203,49.058476
XGBoost_MinMaxScale,0.230531,41.771884,0.035830,49.067973
XGBoost_RobustScale,0.230650,41.768656,0.036294,49.056157
XGBoost_PCA,0.230241,41.779740,0.036613,49.048042
DecisionTree_Base,0.012217,47.328110,0.011627,49.680023
DecisionTree_StandardScale,0.015202,47.256544,0.015045,49.594042
DecisionTree_MinMaxScale,0.014639,47.270066,0.011949,49.671934
DecisionTree_RobustScale,0.023159,47.065257,0.011841,49.674638
DecisionTree_PCA,0.014769,47.266947,0.014489,49.608029


In [86]:
# saving to csv
results_df.to_csv('results.csv')

### Testing final unseen dataset for 1st week Jan 2020

In [62]:
# loading data
test = pd.read_csv('final_test.csv')
# checking 
test.head()

,Unnamed: 0,mkt_carrier_fl_num,origin_airport_id,dest_airport_id,crs_dep_time,crs_arr_time,crs_elapsed_time,distance,mkt_unique_carrier,tail_num,...,day_of_week,mean_taxi_out/time,mean_taxi_in/time,mean_dep_delay/time,mean_arr_delay/time,mean_dep_delay/distance,mean_arr_delay/distance,mean_dep_delay/carrier,mean_arr_delay/carrier,avg_monthly_pas
0,3,5106,14679,12889,1015,1135,80,258,WN,N7828A,...,2,17.522463,6.357388,5.223706,-1.842466,14.347193,11.623701,10.934234,3.841416,76419.0
1,20,6047,14679,13796,2030,2200,90,446,WN,N200WN,...,2,18.684122,7.430636,17.057432,9.755814,7.278261,0.539130,10.934234,3.841416,76419.0
2,44,4011,14679,14771,2155,2330,95,447,WN,N219WN,...,2,15.232919,5.480851,17.018634,9.927660,11.500000,7.128070,10.934234,3.841416,76419.0
3,48,1334,14679,14831,2100,2220,80,417,WN,N7817J,...,2,17.018750,6.824490,22.703125,6.872951,8.224390,4.165854,10.934234,3.841416,76419.0
4,49,1334,14679,14831,2100,2220,80,417,WN,N7817J,...,2,17.018750,6.824490,22.703125,6.872951,8.224390,4.165854,10.934234,3.841416,76419.0


In [63]:
# dropping Unnamed column
test = test.drop(['Unnamed: 0'], axis=1)

# filtering for 1st week of Jan
test = test[test['day'] <= 7]

# checking 
test

,mkt_carrier_fl_num,origin_airport_id,dest_airport_id,crs_dep_time,crs_arr_time,crs_elapsed_time,distance,mkt_unique_carrier,tail_num,branded_code_share,...,day_of_week,mean_taxi_out/time,mean_taxi_in/time,mean_dep_delay/time,mean_arr_delay/time,mean_dep_delay/distance,mean_arr_delay/distance,mean_dep_delay/carrier,mean_arr_delay/carrier,avg_monthly_pas
0,5106,14679,12889,1015,1135,80,258,WN,N7828A,WN,...,2,17.522463,6.357388,5.223706,-1.842466,14.347193,11.623701,10.934234,3.841416,76419.0
1,6047,14679,13796,2030,2200,90,446,WN,N200WN,WN,...,2,18.684122,7.430636,17.057432,9.755814,7.278261,0.539130,10.934234,3.841416,76419.0
2,4011,14679,14771,2155,2330,95,447,WN,N219WN,WN,...,2,15.232919,5.480851,17.018634,9.927660,11.500000,7.128070,10.934234,3.841416,76419.0
3,1334,14679,14831,2100,2220,80,417,WN,N7817J,WN,...,2,17.018750,6.824490,22.703125,6.872951,8.224390,4.165854,10.934234,3.841416,76419.0
4,1334,14679,14831,2100,2220,80,417,WN,N7817J,WN,...,2,17.018750,6.824490,22.703125,6.872951,8.224390,4.165854,10.934234,3.841416,76419.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7898,4086,11433,12339,1752,1910,78,231,DL,N896SK,DL_CODESHARE,...,1,20.287500,7.439678,12.050000,11.778976,7.351759,2.552764,9.613156,2.637875,108812.5
7899,4240,14869,11921,1345,1454,69,216,DL,N779CA,DL_CODESHARE,...,1,15.947154,8.565574,10.206897,8.393443,-0.093284,-1.074906,9.613156,2.637875,94938.5
7900,5514,12451,11292,630,838,248,1447,UA,N145SY,UA_CODESHARE,...,1,16.813246,8.210084,4.561346,-1.672269,31.097561,24.175000,12.628894,8.687159,6818.5
7901,5636,13930,14576,1400,1649,109,528,UA,N912SW,UA_CODESHARE,...,1,19.677898,7.909722,15.648248,8.500000,8.275000,2.815000,12.628894,8.687159,217402.5


In [64]:
# encoding test set
X_blind_test = encoder.transform(test)

# getting predictions from best model (XGBoost, no scaling as xgb1)
test_pred = xgb1.predict(X_blind_test)

# viewing 
test_pred

array([ 1.2833313,  1.3950759,  7.602441 , ..., -4.450466 ,  7.6021485,
        7.6021485], dtype=float32)

In [65]:
# creating column in table
test['predicted_delay'] = test_pred

# checking table
test

,mkt_carrier_fl_num,origin_airport_id,dest_airport_id,crs_dep_time,crs_arr_time,crs_elapsed_time,distance,mkt_unique_carrier,tail_num,branded_code_share,...,mean_taxi_out/time,mean_taxi_in/time,mean_dep_delay/time,mean_arr_delay/time,mean_dep_delay/distance,mean_arr_delay/distance,mean_dep_delay/carrier,mean_arr_delay/carrier,avg_monthly_pas,predicted_delay
0,5106,14679,12889,1015,1135,80,258,WN,N7828A,WN,...,17.522463,6.357388,5.223706,-1.842466,14.347193,11.623701,10.934234,3.841416,76419.0,1.283331
1,6047,14679,13796,2030,2200,90,446,WN,N200WN,WN,...,18.684122,7.430636,17.057432,9.755814,7.278261,0.539130,10.934234,3.841416,76419.0,1.395076
2,4011,14679,14771,2155,2330,95,447,WN,N219WN,WN,...,15.232919,5.480851,17.018634,9.927660,11.500000,7.128070,10.934234,3.841416,76419.0,7.602441
3,1334,14679,14831,2100,2220,80,417,WN,N7817J,WN,...,17.018750,6.824490,22.703125,6.872951,8.224390,4.165854,10.934234,3.841416,76419.0,7.552221
4,1334,14679,14831,2100,2220,80,417,WN,N7817J,WN,...,17.018750,6.824490,22.703125,6.872951,8.224390,4.165854,10.934234,3.841416,76419.0,7.552221
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7898,4086,11433,12339,1752,1910,78,231,DL,N896SK,DL_CODESHARE,...,20.287500,7.439678,12.050000,11.778976,7.351759,2.552764,9.613156,2.637875,108812.5,0.370192
7899,4240,14869,11921,1345,1454,69,216,DL,N779CA,DL_CODESHARE,...,15.947154,8.565574,10.206897,8.393443,-0.093284,-1.074906,9.613156,2.637875,94938.5,-5.927713
7900,5514,12451,11292,630,838,248,1447,UA,N145SY,UA_CODESHARE,...,16.813246,8.210084,4.561346,-1.672269,31.097561,24.175000,12.628894,8.687159,6818.5,-4.450466
7901,5636,13930,14576,1400,1649,109,528,UA,N912SW,UA_CODESHARE,...,19.677898,7.909722,15.648248,8.500000,8.275000,2.815000,12.628894,8.687159,217402.5,7.602149


In [77]:
# re-adding removed origin/destination columns from original table
original_test = pd.read_csv('DB/test_sample.csv', index_col='Unnamed: 0')

# creating dictionary of location names for airport ids
ids = original_test[['origin_airport_id','origin']].drop_duplicates()
ids

,origin_airport_id,origin
0,14679,SAN
81,14683,SAT
125,14771,SFO
126,14831,SJC
189,14843,SJU
...,...,...
73717,14534,RIW
73773,13964,OTH
93976,14222,PPG
102019,12119,HGR


In [78]:
# creating dict
temp = ids.set_index('origin_airport_id',inplace=True)
temp_dict = ids.to_dict()['origin']

# creating new origin column
test['origin'] = test['origin_airport_id'].apply(lambda x: temp_dict.get(x))
# creating new destination column
test['dest'] = test['dest_airport_id'].apply(lambda x: temp_dict.get(x))

# checking
test.head()

,mkt_carrier_fl_num,origin_airport_id,dest_airport_id,crs_dep_time,crs_arr_time,crs_elapsed_time,distance,mkt_unique_carrier,tail_num,branded_code_share,...,mean_dep_delay/time,mean_arr_delay/time,mean_dep_delay/distance,mean_arr_delay/distance,mean_dep_delay/carrier,mean_arr_delay/carrier,avg_monthly_pas,predicted_delay,origin,dest
0,5106,14679,12889,1015,1135,80,258,WN,N7828A,WN,...,5.223706,-1.842466,14.347193,11.623701,10.934234,3.841416,76419.0,1.283331,SAN,LAS
1,6047,14679,13796,2030,2200,90,446,WN,N200WN,WN,...,17.057432,9.755814,7.278261,0.539130,10.934234,3.841416,76419.0,1.395076,SAN,OAK
2,4011,14679,14771,2155,2330,95,447,WN,N219WN,WN,...,17.018634,9.927660,11.500000,7.128070,10.934234,3.841416,76419.0,7.602441,SAN,SFO
3,1334,14679,14831,2100,2220,80,417,WN,N7817J,WN,...,22.703125,6.872951,8.224390,4.165854,10.934234,3.841416,76419.0,7.552221,SAN,SJC
4,1334,14679,14831,2100,2220,80,417,WN,N7817J,WN,...,22.703125,6.872951,8.224390,4.165854,10.934234,3.841416,76419.0,7.552221,SAN,SJC


In [79]:
# checking for null
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7903 entries, 0 to 7902
Data columns (total 27 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   mkt_carrier_fl_num       7903 non-null   int64  
 1   origin_airport_id        7903 non-null   int64  
 2   dest_airport_id          7903 non-null   int64  
 3   crs_dep_time             7903 non-null   int64  
 4   crs_arr_time             7903 non-null   int64  
 5   crs_elapsed_time         7903 non-null   int64  
 6   distance                 7903 non-null   int64  
 7   mkt_unique_carrier       7903 non-null   object 
 8   tail_num                 7903 non-null   object 
 9   branded_code_share       7903 non-null   object 
 10  year                     7903 non-null   int64  
 11  month                    7903 non-null   int64  
 12  week                     7903 non-null   int64  
 13  day                      7903 non-null   int64  
 14  day_of_week             

In [80]:
# filtering columns for submission table
keeping = ['year',
           'month',
           'day',
           'mkt_unique_carrier',
           'mkt_carrier_fl_num',
           'origin',
           'dest',
           'predicted_delay']

submission = test[keeping]

# viewing
submission

,year,month,day,mkt_unique_carrier,mkt_carrier_fl_num,origin,dest,predicted_delay
0,2020,1,1,WN,5106,SAN,LAS,1.283331
1,2020,1,1,WN,6047,SAN,OAK,1.395076
2,2020,1,1,WN,4011,SAN,SFO,7.602441
3,2020,1,1,WN,1334,SAN,SJC,7.552221
4,2020,1,1,WN,1334,SAN,SJC,7.552221
...,...,...,...,...,...,...,...,...
7898,2020,1,7,DL,4086,DTW,IND,0.370192
7899,2020,1,7,DL,4240,SLC,GJT,-5.927713
7900,2020,1,7,UA,5514,JAX,DEN,-4.450466
7901,2020,1,7,UA,5636,ORD,ROC,7.602149


In [81]:
# saving to csv
submission.to_csv('submission.csv')